In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os 
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.conn()

In [ ]:
from datetime import datetime
from pathlib import Path
from workflow.pipeline import ephys, probe, induction, lineage, analysis

In [ ]:
dj.Diagram(induction) + dj.Diagram(lineage) + dj.Diagram(ephys) + dj.Diagram(probe) + dj.Diagram(analysis)

In [ ]:
ephys.LFP.Trace()

In [ ]:
lfp_traces, ti, tf, electrodes = ephys.LFP.Trace.fetch("lfp", "start_time", "end_time", "electrode")
sampling_rate = ephys.LFP.fetch1("lfp_sampling_rate")

idx = np.argsort(electrodes)

labels = np.array([f'D-0{x}' for x in electrodes])

labels = labels[idx]
lfp_traces = lfp_traces[idx]
electrodes = electrodes[idx]

# plot all channels
fig, ax = plt.subplots(figsize=(20, 10))
ytick_loc = []
for i, (lfp_trace, electrode) in enumerate(zip(lfp_traces, electrodes)): 
    ytick_loc.append(i * 500)
    ax.plot(np.r_[:len(lfp_trace)]/sampling_rate, lfp_trace + i * 500 , linewidth=0.1)
ax.set_yticks(ytick_loc)
ax.set_yticklabels([f'D-0{x}' for x in electrodes])
ax.tick_params(length=0)
ax.set(xlabel="Time (s)")
sns.despine(right = True, left = True)
plt.show()

In [ ]:
Sxx, t, f = analysis.LFPSpectrogram.ChannelSpectrogram.fetch("spectrogram", "time", "frequency", limit=1)
Sxx = Sxx[0]
t = t[0]
f = f[0]

fig, ax = plt.subplots(figsize=(20, 5))
ax.pcolormesh(t, f[1:1000], np.log(Sxx[1:1000]), shading='gouraud')
plt.yscale("log")
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
analysis.LFPSpectrogram.Power()